# CloudPerformance - Data Parser

## [DOWNLOAD DATA HERE](https://mega.nz/folder/lMFTkQLK#uFvAUKTCT2jDfO0mgs5ZPw)

In [ ]:
provider = "aws"

In [ ]:
provider = "azure"

In [ ]:
provider = "gcp"

In [ ]:
source = "local"
source_file = provider + ".json"
config_file = "config_data_parser_" + provider + ".yml"
ignore_vms_list = []

### Data parsing and analysis
- the suite of benchmarks is repeated at every interval of time
- a (micro)benchmark can be repeated multiple times
- a (micro)benchmark can have multiple setup
- the benchmark time is taken when the suite of benchmark is completed

In [ ]:
import yaml
import requests
import matplotlib.pyplot as plt
import statistics as stat
import json
import datetime
import re
import pandas as pd
import numpy as np
import os
import plotly.graph_objects as go
import csv
from matplotlib.dates import DateFormatter
import pickle
import hashlib 

In [ ]:
# set plot format
%matplotlib inline
plt.rcParams["figure.figsize"] = (20,10)
cmap = plt.get_cmap('jet_r')
font = {'family' : 'DejaVu Sans',
        'weight' : 'normal',
        'size'   : 18}
plt.rc('font', **font)

OUTPUT_DIRS = ["output/execution/html/", "output/execution/img/", "output/time/html/", "output/time/img/", "output/bar/html/", "output/bar/img/", "output/delta/html/", "output/delta/img/", "pickle/"]
for output_dir in OUTPUT_DIRS:
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

# set date formatter
date_formatter = DateFormatter('%H:%M')        

def get_valid_filename(s):
    s = str(s).strip().replace(' ', '_')
    return re.sub(r'(?u)[^-\w.]', '', s)

In [ ]:
# helpers
def plot_dates(x, y, label=None, suptitle=None):
    fig, ax = plt.subplots()
    if suptitle:
        fig.suptitle(suptitle)
    ax.plot(x, y, label=label)
    ax.xaxis.set_major_formatter(date_formatter)
    fig.autofmt_xdate()
    plt.grid(True, linewidth=0.3, linestyle='-')
    plt.show()
    plt.close()
    
def plotly_dates(x, y, label=None, suptitle=None):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name=label))
    fig.update_layout(autosize=False)
    fig.update_yaxes(automargin=False)
    fig.show()

### Parsers Definition
Extract useful data from benchmarks outputs using regexps

In [ ]:
def parser_sysbench(raw_data, print_errors=True):
    sys_res = []
    cpu_tests = [(r"events per second:\s*([0-9.]*)", "cpu: events per second", "events/s"),
                (r"avg:\s*([0-9.]*)", "cpu: avg latency", "ms")]
    fileio_tests = [(r"reads\/s:\s*([0-9.]*)", "fileio: fileop - read", "reads/s"),
                    (r"writes\/s:\s*([0-9.]*)", "fileio: fileop - write", "writes/s"),
                    (r"fsyncs\/s:\s*([0-9.]*)", "fileio: fileop - fsync", "fsyncs/s"),
                    (r"read, MiB\/s:\s*([0-9.]*)", "fileio: throughput - read", "MiB/s"),
                    (r"written, MiB\/s:\s*([0-9.]*)", "fileio: throughput - write", "MiB/s"),
                    (r"avg:\s*([0-9.]*)", "fileio: avg latency", "ms")]
    memory_tests = [(r"([0-9.,]*) per second", "memory: totalops per second", "ops/s"),
                    (r"([0-9.,]*) MiB\/sec", "memory: speed", "MiB/s"),
                    (r"avg:\s*([0-9.]*)", "memory: avg latency", "ms")]
    threads_tests = [(r"avg:\s*([0-9.]*)", "threads: avg latency", "ms")]
    for benchmark in [("cpu", cpu_tests), ("fileio", fileio_tests),
                      ("memory", memory_tests), ("threads", threads_tests)]:
        r_data = raw_data[benchmark[0]]
        tests = benchmark[1]
        for test in tests:
            match = re.findall(test[0], r_data, re.MULTILINE)
            res = float(match[0].replace(",", "."))
            name = test[1]
            unit = test[2]
            sys_res.append({"unit": unit, "value": res, "name": name})
    return sys_res

def parser_simplecpu(raw_data, print_errors=True):
    return [{"unit": "s", "value": raw_data, "name": "mean duration"}]

def parser_download(raw_data, print_errors=True):
    # extract size and time
    regex = r"Downloaded\s([0-9]*)[a-z\s]*([0-9.,]*)"
    match = re.findall(regex, raw_data, re.MULTILINE)
    size = float(match[0][0])/1000/1000
    time = float(match[0][1].replace(",", "."))
    return [{"unit": "MB/s", "value": round(size/time, 2), "name": "speed"}]

def parser_dd(raw_data, print_errors=True):
    # extract speed
    regex = r"s,\s([0-9.,]*)\s[a-zA-Z]*\/s"
    match = re.findall(regex, raw_data, re.MULTILINE)
    speed = float(str(match[0]).replace(",", "."))
    return [{"unit": "MB/s", "value": speed, "name": "speed"}]

def parser_web_benchmark(raw_data, print_errors=True):
    regex = r"Requests/sec:\s*([0-9.]*)"
    match = re.findall(regex, raw_data, re.MULTILINE)
    speed = float(str(match[0]).replace(",", "."))
    return [{"unit": "req/s", "value": speed, "name": "requests per second"}]

def parser_nench(raw_data, print_errors=True):
    # cpu
    try:
        cpu_res = []
        for cpu_test in ["SHA256", "bzip2", "AES"]:
            regex = re.escape(cpu_test) + ".*\n\s*([0-9,.]*) ([a-z]*)"
            match = re.findall(regex, raw_data, re.MULTILINE)
            res = float(str(match[0][0]).replace(",", "."))
            unit = str(match[0][1])
            cpu_res.append({"unit": unit, "value": res, "name": "cpu - " + cpu_test})
    except Exception as e:
        if print_errors:
            print("error: nench - cpu: " + str(e))
    # ioping
    try:
        ioping_res = []
        seekrate = (r"min\/avg\/max\/mdev = [0-9a-z.\s]*\/ ([0-9.]*) ([a-z]*)", "ioping: avg seek rate")
        srs = (r"generated.*iops. ([0-9.]*) (.*)", "ioping: sequential read speed")
        for ioping in [seekrate, srs]:
            regex = ioping[0]
            match = re.findall(regex, raw_data, re.MULTILINE)
            res = float(str(match[0][0]).replace(",", "."))
            unit = str(match[0][1])
            ioping_res.append({"unit": unit, "value": res, "name": ioping[1]})
    except Exception as e:
        if print_errors:
            print("error: nench - ioping: " + str(e))
        pass
    # dd
    try:
        dd_res = []
        regex = r"average:\s*([0-9.]*) ([a-zA-Z\/]*)$"
        match = re.findall(regex, raw_data, re.MULTILINE)
        res = float(str(match[0][0]).replace(",", "."))
        unit = str(match[0][1])
        dd_res.append({"unit": unit, "value": res, "name": "dd: avg sequential write speed"})
    except Exception as e:
        if print_errors:
            print("error: nench - dd: " + str(e))
    # speed test
    try:
        speed_res = []
        for speed_test in ["Cachefly CDN", "Leaseweb (NL)", "Softlayer DAL (US)", "Online.net (FR)", "OVH BHS (CA)"]:
            regex = re.escape(speed_test) + ":\s*([0-9.]*) ([a-zA-Z\/]*)"
            match = re.findall(regex, raw_data, re.MULTILINE)
            res = float(str(match[0][0]).replace(",", "."))
            unit = str(match[0][1])
            speed_res.append({"unit": unit, "value": res, "name": "net speed - " + speed_test})
    except Exception as e:
        if print_errors:
            print("error: nench - speed test: " + str(e))
    return cpu_res + ioping_res + dd_res + speed_res

def parser_aibenchmark(raw_data, print_errors=True):
    ai_res = []
    if raw_data["ai_score"]:
        ai_res.append({"unit": "score", "value": raw_data["ai_score"], "name": "ai score"})
    if raw_data["inference_score"]:
        ai_res.append({"unit": "score", "value": raw_data["inference_score"], "name": "inference score"})
    if raw_data["training_score"]:
        ai_res.append({"unit": "score", "value": raw_data["training_score"], "name": "training score"})
    return ai_res

def htr(unit):
    if unit == "s" or unit == "seconds" or unit =="ms" or unit == "us":
        return "LIB"
    return "HIB"

### Apply parsers and extract data
For each benchmark extract a list of values (a benchmark can provide multiple extracted values, such as sysbench):
- extracted.name: is the name of the benchmark value extracted, e.g. cpu - SHA256
- extracted.value: is the extracted value, e.g. 3.279
- extracted.unit: extracted unit, e.g. seconds

In [ ]:
benchmark_extractors = {"download": parser_download,
                        "dd": parser_dd,
                        "web-benchmark": parser_web_benchmark,
                        "nench-benchmark": parser_nench,
                        "sys-benchmark": parser_sysbench,
                        "simple-cpu": parser_simplecpu,
                        "ai-benchmark": parser_aibenchmark}


# get configuration
with open(config_file, 'r') as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)

if source == "local":
    with open(source_file, 'r') as json_file:
        data = json.load(json_file)
elif source == "remote":
    # get data from the database
    url = cfg["bin_database_url"] + "/all"
    # print("getting data from: " + url)
    response = requests.get(url=url)
    data = response.json()

print("total VMs: %d\n\n" % len(data["bins"]))

durations_all = []
benchmarks_all = 0
# iter vms
for bin in data["bins"]:
    pars_err = 0
    id = bin["id"]
    if id in ignore_vms_list:
        continue
    updated = bin["updated"]
    benchmarks = len(bin["values"])
    server = bin["values"][0]["server"]
    
    print(id)
    print(json.dumps(server, indent=2))
    
    # iters bench runs
    durations = []
    times = []
    results = {}
    for value in bin["values"]:
        durations_all.append(value["duration"])
        durations.append(value["duration"])
        time = datetime.datetime.strptime(value["time"], '%Y-%m-%d %H:%M:%S.%f')
        times.append(time)
        
        # iter benchmarks
        for benchmark in value["benchmarks"]:
            if benchmark["name"] in benchmark_extractors.keys():
                try:
                    benchmark["extracted"] = benchmark_extractors[benchmark["name"]](benchmark["result"]["output"], print_errors=False)
                    benchmarks_all += 1
                except Exception as e:
                    print("Error extracting: " + benchmark["name"] + " @ " + str(time) + "\n" + str(e))
                    pars_err += 1
    print("Parsing errors: " + str(pars_err))
        
    print("total runs: %d\nupdated: %s\nduration (min, avg, max): %.2f m, %.2f m, %.2f m"
          % (benchmarks, updated, min(durations)/60, stat.mean(durations)/60, max(durations)/60))
    
    fig = go.Figure([go.Scatter(x=times, y=durations)])
    fig.update_layout(
        title=provider + " / " + cfg["vms"][id.split("_", 1)[1]]["name"],
        title_x=0.5,
        xaxis_title="time",
        yaxis_title="duration",
    )
    fig.write_image("output/execution/img/" + id + ".png", scale=2)
    fig.update_xaxes(rangeslider_visible=True)
    fig.write_html("output/execution/html/" + id + ".html", include_plotlyjs="cdn")
    fig.show()

print("OVERALL benchmarks: %d\nduration (min, avg, max): %.2f m, %.2f m, %.2f m"
          % (benchmarks_all, min(durations_all)/60, stat.mean(durations_all)/60, max(durations_all)/60))
# print(json.dumps(data, indent=2))

## Data analysis
- use Pandas to analyze data
- preparare the data for Pandas: build a list where each entry is a benchmark value

In [ ]:
rows = []
tests_uni = {}
# iter vms
for bin in data["bins"]:
    id = bin["id"]
    if id in ignore_vms_list:
        continue
    updated = bin["updated"]
    server = bin["values"][0]["server"]
    
    # iters bench run
    for value in bin["values"]:
        duration = value["duration"]
        time = value["time"]
        
        # iter benchmarks
        for benchmark in value["benchmarks"]:
            if "extracted" not in benchmark:
                break
            extracted = benchmark["extracted"]
            
            for ext in extracted:
                test_id = hashlib.sha1(str(str(benchmark["name"] + str(benchmark["setup"]) + ext["name"])).encode()).hexdigest()
                row = {"id": cfg["vms"][id.split("_", 1)[1]]["name"],
                       "duration": duration,
                       "time": pd.to_datetime(time, format='%Y-%m-%d %H:%M:%S.%f'),
                       "value": ext["value"],
                       "test_id": test_id}
                
                tests_uni[test_id] = {"bench_name": benchmark["name"],
                                      "setup": str(benchmark["setup"]),
                                      "extracted_name": ext["name"],
                                      "extracted_unit": ext["unit"],
                                      "htr": htr(ext["unit"]) }
                
                rows.append(row)
                
# save tests
pickle.dump(tests_uni, open("pickle/tests_uni.p", "wb"))

#### Print the tests

In [ ]:
print(json.dumps(tests_uni, indent=2))
print("\nBenchmark list (" + str(len(tests_uni)) + "):")
for test in tests_uni:
    print(tests_uni[test]["bench_name"], tests_uni[test]["extracted_name"], tests_uni[test]["extracted_unit"], tests_uni[test]["htr"])

#### Print the dataset

In [ ]:
print(len(rows))
# normalize the data
pd.set_option('display.max_rows', 20)
df = pd.json_normalize(rows)
df

In [ ]:
df.describe()

#### Print the extracted values for each benchmark with all the machines in a single graph
1. compute the mean for each benchmark for each machine grouping by: id, test_id, time
2. reset_index() to obtain a list of tuples
3. groupby bench_name, setup, extracted.name
4. plot for each group, grouping by id

In [ ]:
pd.set_option('display.max_rows', 20)
gb = df.groupby(['id', 'test_id', 'time']).agg(
    {'value': ['mean']}).reset_index()
gb.columns = gb.columns.droplevel(1)
gb

In [ ]:
pd.set_option('display.max_rows', None)
gbb = gb.groupby(['test_id'])
# save gbb
pickle.dump(gbb, open("pickle/" + provider + ".p", "wb"))
gbb.first() # print the first entry (can be many)

### Time/Result Graph

In [ ]:
def get_filename(provider, bench_name, extracted_name, key):
    return provider + "_" + get_valid_filename(bench_name + "_" + extracted_name) + "_" + str(key)[0:5]
def get_trace_name(name):
    if provider == "aws":
        #return name.replace("-central", "")
        return name[:name.find(" -")]+"-"+name[-1:]
    elif provider == "azure":
        return name
    elif provider == "gcp":
        return name

In [ ]:
for key, grp in gbb:
    print(tests_uni[key])
    print("\n")
    fig = go.Figure()
    gbid = grp.groupby("id")
    for key2, grp2 in gbid:
        times = grp2["time"]
        values = grp2["value"]
        fig.add_trace(go.Scatter(x=times, y=values, mode='lines', name=get_trace_name(key2)))
    fig.update_layout(autosize=True)
    fig.update_yaxes(automargin=True)
    fig.update_layout(
        title=provider + " / " + tests_uni[key]["bench_name"] + " - " + tests_uni[key]["extracted_name"] + " (" + tests_uni[key]["extracted_unit"] + ")",
        title_x=0.5,
        xaxis_title="time",
        yaxis_title="result",
    )
    fig.show()
    fig.write_html("output/time/html/" + get_filename(provider, tests_uni[key]["bench_name"], tests_uni[key]["extracted_name"], key) + ".html", include_plotlyjs="cdn")
    fig.write_image("output/time/img/" + get_filename(provider, tests_uni[key]["bench_name"], tests_uni[key]["extracted_name"], key) + ".png", scale=2)
    print("\n\n")

### Time/Result Bar Graph

In [ ]:
for key, grp in gbb:
    print(tests_uni[key])
    print("\n")
    fig = go.Figure()
    gbid = grp.groupby("id")
    for key2, grp2 in gbid:
        print("%s - \tAVG: %.2f, STD: %.2f, MIN: %.2f, MAX: %.2f" % (key2, grp2.value.mean(), grp2.value.std(), grp2.value.min(), grp2.value.max()))
        fig.add_trace(go.Bar(
            name=get_trace_name(key2),
            x=[get_trace_name(key2)], y=[grp2.value.mean()],
            error_y=dict(type='data', array=[grp2.value.std()])
        ))
    fig.update_layout(autosize=True)
    fig.update_yaxes(automargin=True)
    fig.update_layout(
        title=provider + " / " + tests_uni[key]["bench_name"] + " - " + tests_uni[key]["extracted_name"] + " (" + tests_uni[key]["extracted_unit"] + ")",
        title_x=0.5,
        xaxis_title="instance",
        yaxis_title="result",
        showlegend=False
    )
    fig.show()
    fig.write_html("output/bar/html/" + get_filename(provider, tests_uni[key]["bench_name"], tests_uni[key]["extracted_name"], key) + ".html", include_plotlyjs="cdn")
    fig.write_image("output/bar/img/" + get_filename(provider, tests_uni[key]["bench_name"], tests_uni[key]["extracted_name"], key) + ".png", scale=2)
    print("\n\n")

### Time/Result Delta Graph

In [ ]:
for key, grp in gbb:
    print(tests_uni[key])
    print("\n")
    fig = go.Figure()
    gbid = grp.groupby("id")
    for key2, grp2 in gbid:
        times = grp2["time"]
        values = [v/grp2.value.mean()*100 for v in grp2["value"]]
        fig.add_trace(go.Scatter(x=times, y=values, mode='lines', name=get_trace_name(key2)))
        print("%s - \tAVG: %.2f, STD: %.2f, MIN: %.2f, MAX: %.2f" % (key2, stat.mean(values), stat.stdev(values), min(values), max(values)))
    fig.update_layout(autosize=True)
    fig.update_yaxes(automargin=True)
    fig.update_layout(
        title=provider + " / " + tests_uni[key]["bench_name"] + " - " + tests_uni[key]["extracted_name"] + " (" + tests_uni[key]["extracted_unit"] + ")",
        title_x=0.5,
        xaxis_title="time",
        yaxis_title="Δ",
    )
    fig.show()
    fig.write_html("output/delta/html/" + get_filename(provider, tests_uni[key]["bench_name"], tests_uni[key]["extracted_name"], key) + ".html", include_plotlyjs="cdn")
    fig.write_image("output/delta/img/" + get_filename(provider, tests_uni[key]["bench_name"], tests_uni[key]["extracted_name"], key) + ".png", scale=2)
    print("\n\n")